In [253]:
%set_env PROJECT_PATH=/Users/gclyne/thesis
import os 
from functools import reduce
import xarray as xr
import config as config
import pandas as pd
import plotly.graph_objects as go
from pickle import load
cesm_data = pd.read_csv('/Users/gclyne/thesis/data/cesm_data.csv')
os.chdir(f'{config.DATA_PATH}/generated_observable_data')
files = os.listdir()
data_frames = []
for file in files:
    print(file)
    data_frames.append(pd.read_csv(file))
observed_data = reduce(lambda  left,right: pd.merge(left,right,on=['year','lat','lon'],
                                            how='outer'), data_frames)
cesm_data['grass_crop_shrub'] = cesm_data['cropFrac'] + cesm_data['shrubFrac'] + cesm_data['grassFrac']
observed_data['total_lai'] = observed_data['leaf_area_index_low_vegetation.nc'] + observed_data['leaf_area_index_high_vegetation.nc']
#('pr','tas','# lai','treeFrac','baresoilFrac','ps','grass_crop_shrub')
observed_data['total_precipitation.nc'] = observed_data['total_precipitation.nc'] / (24*60*60*30.437) * 1000
years = [x for x in range(1984,2015)]
observed_data = observed_data.groupby('year').mean()
final_input = observed_data[['total_precipitation.nc','2m_temperature.nc','total_lai','observed_tree_cover',
'observed_bare','surface_pressure.nc','observed_shrub_bryoid_herb']]

env: PROJECT_PATH=/Users/gclyne/thesis
era_leaf_area_index_low_vegetation_data.csv
era_surface_pressure_data.csv
era_total_precipitation_data.csv
era_surface_net_solar_radiation_data.csv
era_leaf_area_index_high_vegetation_data.csv
era_2m_temperature_data.csv
nfis_tree_cover_data.csv


In [254]:
scaler = load(open('/Users/gclyne/thesis/data/scaler.pkl', 'rb'))
final_input.columns = ['pr','tas','# lai','treeFrac','baresoilFrac','ps','grass_crop_shrub']
final_input = scaler.transform(final_input)


In [255]:

columns = ['pr','tas','# lai','treeFrac','baresoilFrac','ps','grass_crop_shrub']
for index in range(len(columns)):
    cesm_data[columns[index]]
    final_input[:,index]
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=years, y=final_input[:,index],
                        mode='lines',
                        name=f'Observed {columns[index]}'))
    fig.add_trace(go.Scatter(x=years, y=cesm_data[columns[index]][years],
                        mode='markers',
                        name=f'CESM {columns[index]}'))

    fig.update_layout(legend_title_text = "Variable")
    fig.update_xaxes(title_text="year")
    fig.update_yaxes(title_text="measurement")
    fig.write_image(f'/Users/gclyne/thesis/{columns[index]}_cesm_observed_comparison.png')


In [34]:
lai_observed = []
from numpy import genfromtxt
for year in range(1984,2015,1):
    my_data = genfromtxt(f'/Users/gclyne/thesis/data/modis_lai_data_{year}', delimiter=',',skip_header=True)
    lai_observed.append(my_data[:,0].mean()* 0.1) #0.1 scaling factor per the documentation


In [56]:
import pandas as pd
temp = lai_yearly.lai.values[-31:]
lai_df = pd.DataFrame(lai_observed,temp)

# px.scatter(lai_df,x=)
# \
import plotly.graph_objects as go


years = [x for x in range(0,31)]
# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=years, y=temp,
                    mode='lines',
                    name='lines'))
fig.add_trace(go.Scatter(x=years, y=lai_observed,
                    mode='lines+markers',
                    name='lines+markers'))

fig.show()


In [146]:

years = [x for x in range(1984,2019)]
fig = go.Figure()
soil_colour = 'brown'
cwd_colour = 'blue'
veg_colour = 'green'
litter_colour = 'yellow'
fig.add_trace(go.Scatter(x=years, y=predicted_forest_carbon['cSoil'],
                    mode='markers',
                    name='predicted soil',
                    marker=dict(color=soil_colour)
                ))
fig.add_trace(go.Scatter(x=years, y=cesm_data['cSoil'] ,
                    mode='lines',
                    name='cesm soil',
                    marker=dict(color=soil_colour)
))

fig.add_trace(go.Scatter(x=years, y=predicted_forest_carbon['cCwd'],
                    mode='markers',
                    name='predicted cwd',
                    marker=dict(color=cwd_colour)
                ))
fig.add_trace(go.Scatter(x=years, y=cesm_data['cCwd'] ,
                    mode='lines',
                    name='cesm cwf',
                    marker=dict(color=cwd_colour)
))

fig.add_trace(go.Scatter(x=years, y=predicted_forest_carbon['cVeg'],
                    mode='markers',
                    name='predicted veg',
                    marker=dict(color=veg_colour)
                ))
fig.add_trace(go.Scatter(x=years, y=cesm_data['cVeg'] ,
                    mode='lines',
                    name='cesm veg',
                    marker=dict(color=veg_colour)
))


fig.update_layout(legend_title_text = "Variable")
fig.update_xaxes(title_text="year")
fig.update_yaxes(title_text="kg/m2")
fig.show()

In [152]:
predicted_forest_carbon

# 80kg/m2 -> managed forest is 230,000 kha 
# 1 hectare = 10,000m^2
# 1kha = 10,000,000m^2
# 230,000 * 10,000,000 = 2.3e+12m^2 == managed forest area 
# 80 * 2.3e+12 = 184000000000000kg carbon in managed forest area
#1 megatonne = 1 000 000 000
# 184000000000 tonnes

1.61e+13 / 1000000000


16100.0

In [20]:
# actual size of managed forest 4406620292520.00000000000
# size that gridded cells give 4335325524787.99
#gridded gives  71294767732.00977 less that actual, approx 1.61% less area
4335325524787.99 * .58 / 10000

251448880.43770343